<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [16]</a>'.</span>

In [ ]:
import datetime as dt
import glob
import math
from pathlib import Path
import warnings

import folium
import geopandas as gpd
import h5py
from isce3.core import Ellipsoid as ellips
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyproj import Proj, CRS
import pysolid
import requests
import shapely.wkt as wkt
from shapely import geometry

from src.ALE_utils import oversample_slc, findCR, interpolate_correction_layers
warnings.filterwarnings('ignore')

# 0. Read OPERA Coregistered SLC

In [ ]:
# Parameters for papermill
wdir = '/mnt/trappist-r0/bato/work/ROSAMOND/COMPASS_TEST/A064/stack/'
burst_id = 't064_135523_iw2'
date = '20221109'

In [ ]:
# Parameters
wdir = "/mnt/trappist-r0/bato/work/ROSAMOND/COMPASS_TEST/A064/stack/"
burst_id = "t064_135523_iw2"
date = "20150207"


In [ ]:
pol = 'VV'
path_h5 = f'{wdir}/{burst_id}/{date}/{burst_id}_{date}.h5'

if not Path(path_h5).exists():
    raise SystemExit(f'Specified file {path_h5} does not exist, exit notebook')


In [ ]:
# Load the CSLC and necessary metadata
DATA_ROOT = 'science/SENTINEL1'
grid_path = f'{DATA_ROOT}/CSLC/grids'
metadata_path = f'{DATA_ROOT}/CSLC/metadata'
burstmetadata_path = f'{DATA_ROOT}/CSLC/metadata/processing_information/s1_burst_metadata'
id_path = f'{DATA_ROOT}/identification'

with h5py.File(path_h5,'r') as h5:
    cslc = h5[f'{grid_path}/{pol}'][:]
    xcoor = h5[f'{grid_path}/x_coordinates'][:]
    ycoor = h5[f'{grid_path}/y_coordinates'][:]
    dx = h5[f'{grid_path}/x_spacing'][()].astype(int)
    dy = h5[f'{grid_path}/y_spacing'][()].astype(int)
    epsg = h5[f'{grid_path}/projection'][()].astype(int)
    sensing_start = h5[f'{burstmetadata_path}/sensing_start'][()].astype(str)
    sensing_stop = h5[f'{burstmetadata_path}/sensing_stop'][()].astype(str)
    dims = h5[f'{burstmetadata_path}/shape'][:]
    bounding_polygon =h5[f'{id_path}/bounding_polygon'][()].astype(str) 
    orbit_direction = h5[f'{id_path}/orbit_pass_direction'][()].astype(str)


In [ ]:
# Visualize burst location on a map
cslc_poly = wkt.loads(bounding_polygon)
bbox = [cslc_poly.bounds[0], cslc_poly.bounds[2], cslc_poly.bounds[1], cslc_poly.bounds[3]]
m = folium.Map(location=[np.mean(bbox[2:3]), np.mean(bbox[0:1])], zoom_start=9, tiles='CartoDB positron')
gdf_cslc = gpd.GeoDataFrame(index=[0], crs=f'epsg:{epsg}', geometry=[cslc_poly])
geoj_cslc = gdf_cslc.to_json()
geoj_cslc = folium.GeoJson(data=geoj_cslc,
                        style_function=lambda x: {'fillColor': 'orange'})
geoj_cslc.add_to(m)
m

In [ ]:
# Visualize CSLC
%matplotlib widget
#bbox = [xcoor.min(),xcoor.max(),ycoor.min(),ycoor.max()]
fig, ax = plt.subplots(figsize=(12, 6))
ax.imshow(20*np.log10(np.abs(cslc)), cmap='gray',interpolation=None, extent=bbox)
fig.suptitle(f'{burst_id}_{date}_{pol}')
ax.set_aspect(1)
ax.set_xlabel('Longitude (deg)')
ax.set_ylabel('Latitude (deg)')
fig.savefig('cslc.png',dpi=300,bbox_inches='tight')

# 1. Get corner reflectors data

In [ ]:
# Get the cslc date
date_ = dt.datetime.strptime(sensing_start.astype(str),'%Y-%m-%d %H:%M:%S.%f').strftime('%Y-%m-%d+%H\u0021%M')

# Download corner reflector data from NISAR based on the date of the CSLC product
res = requests.get(f'https://uavsar.jpl.nasa.gov/cgi-bin/corner-reflectors.pl?date={str(date_)}&project=alaska')
open(f'crdata_{date}.csv', 'wb').write(res.content)

# Read to pandas dataframe and rename columns
df = pd.read_csv(f'crdata_{date}.csv')
df.rename(columns={'Corner reflector ID':'ID'}, inplace=True)
df.rename(columns={'Latitude (deg)':'lat'}, inplace=True) 
df.rename(columns={'Longitude (deg)':'lon'}, inplace=True) 
df.rename(columns={'Azimuth (deg)':'azm'}, inplace=True)
df.rename(columns={'Tilt / Elevation angle (deg)':'tilt'}, inplace=True)
df.rename(columns={'Height above ellipsoid (m)':'hgt'}, inplace=True) 
df.rename(columns={'Side length (m)':'slen'}, inplace=True)

df.head()

In [ ]:
if 'lon' not in df.keys():
    raise SystemExit('No CRs found for this time, exit notebook')

In [ ]:
#calculating the locations of CRs in SAR image
UTMx = []
UTMy = []
xindex = []
yindex = []
xloc = []
yloc = []
_in = []

for idx, row in df.iterrows():
    
    _Proj = Proj(CRS.from_epsg(epsg))
    _x, _y = _Proj(row['lon'], row['lat'],inverse=False)     #conversion of lat/lon of CRs to UTM coordinates
    
    #Indices of the CRs in SLC image
    _xloc = int((_x-xcoor[0])/dx)    
    _yloc = int((_y-ycoor[0])/dy)
    
    UTMx.append(_x) 
    UTMy.append(_y)
    xindex.append(_xloc)
    yindex.append(_yloc)
    xloc.append((_x-xcoor[0])/dx)
    yloc.append((_y-ycoor[0])/dy)
    _in.append(cslc_poly.contains(geometry.Point(row['lon'],row['lat'])))
    
df['UTMx'] = UTMx
df['UTMy'] = UTMy
df['CR_xindex'] = xindex
df['CR_yindex'] = yindex
df['CR_xloc'] = xloc
df['CR_yloc'] = yloc
df['inPoly'] = _in

#checking whether CRs are in SLC coverage. Including only CRs within SLC image
df = df[df['inPoly']==True]
df.drop('inPoly', axis=1, inplace=True)
df = df.reset_index(drop=True)
    
if df.empty:
    raise SystemExit(f'No CRs found within burst for date {date}, exit notebook')
    
df.head()

In [ ]:
dateformat = '%Y-%m-%d %H:%M:%S.%f'  #date format of input azimuth time
dt0 = dt.datetime.strptime(sensing_start,dateformat)
dt1 = dt.datetime.strptime(sensing_stop,dateformat)
step_sec = 5                        # sample spacing in time domain in seconds

for idx, row in df.iterrows():
    
    llh = [np.deg2rad(row['lon']), np.deg2rad(row['lat']), row['hgt']]  #lon/lat/hgt
    
    _elp = ellips()
    xyz = _elp.lon_lat_to_xyz(llh) #xyz coordinate of CR
    
    # compute SET via pysolid
    (dt_out,
     tide_e,
     tide_n,
     tide_u) = pysolid.calc_solid_earth_tides_point(np.rad2deg(llh[1]), np.rad2deg(llh[0]), dt0, dt1,
                                                step_sec=step_sec,
                                                display=False,
                                                verbose=False)
    
    tide_e = np.mean(tide_e[0:2])
    tide_n = np.mean(tide_n[0:2])
    tide_u = np.mean(tide_u[0:2])
    
    #updating lat,lon,hgt after SET correction
    xyz = [xyz[0]+tide_e, xyz[1]+tide_n, xyz[2]+tide_u]
    llh = _elp.xyz_to_lon_lat(xyz)  
    
    df.loc[idx,'lat'] = np.rad2deg(llh[1])
    df.loc[idx,'lon'] = np.rad2deg(llh[0])
    df.loc[idx,'hgt'] = llh[2]   

In [ ]:
#Displaying SLC image
buffer = 50
minX = df['CR_xindex'].min() - buffer
maxX = df['CR_xindex'].max() + buffer
minY = df['CR_yindex'].min() - buffer
maxY = df['CR_yindex'].max() + buffer

scale_ = 1.0
exp_ = 0.15

fig, ax = plt.subplots(figsize=(12, 3))
cax=ax.imshow(scale_*(np.abs(cslc))**exp_, cmap='gray',interpolation=None, origin='upper')
ax.set_xlim(minX,maxX)
ax.set_ylim(minY,maxY)
#ax.axis('off')

for sl in pd.unique(df.slen):
    xx = df.loc[df['slen']==sl]['CR_xindex']
    yy = df.loc[df['slen']==sl]['CR_yindex']
    ID = df.loc[df['slen']==sl]['ID']
    
    if sl == 2.4384:
        color='blue'
    elif sl == 4.8:
        color='red'
    elif sl == 2.8:
        color='yellow'
    else:
        color='green'
    
    ax.scatter(xx,yy,color=color,marker="+",lw=1)
    for _ID,_xx,_yy in zip(ID,xx,yy):
        ax.annotate(_ID, (_xx, _yy), fontsize=10)

ax.set_aspect(1)
fig.savefig('S1_geoSLC_CRs.png',dpi=300,bbox_inches='tight')

In [ ]:
# selecting CRs according to orbit direction
if orbit_direction == 'Descending':
    # only east-looking CRs (for right-looking descending)
    df_filter = df[df['azm']<20].reset_index(drop=True)
else:  #ASCENDING
    # only west-looking CRs (for right-looking ascending)
    df_filter = df.loc[(df['azm']>160) & (df['azm']<200)]
    # reject N32A if data from past 20220601 as it got overturned
    if dt.datetime.strptime('20220601', '%Y%m%d') <= dt.datetime.strptime(str(date), '%Y%m%d'):
        df_filter = df_filter[df_filter['ID']!='N32A'].reset_index(drop=True)

df_filter = df_filter.loc[df_filter['slen']>0.8].reset_index(drop=True)   #excluding SWOT CRs (0.7 m as a side length)
df = None

if df_filter.empty:
    raise SystemExit(f'No valid CRs found within burst for date {date}, exit notebook')

In [ ]:
xpeak = []
ypeak = []

for ID, xoff, yoff in zip(df_filter['ID'],df_filter['CR_xindex'],df_filter['CR_yindex']):
    # crop a patch of 10*10 with center at the calculated CR position
    pxbuff = 5
    pybuff = 5
    cropcslc = cslc[(yoff-pybuff):(yoff+pybuff+1),(xoff-pxbuff):(xoff+pxbuff+1)]
    
    # find the peak amplitude in the 10*10 patch
    yind,xind = np.unravel_index(np.argmax(np.abs(cropcslc), axis=None), cropcslc.shape)
    
    # give a warning if the peak and the calculated postion are too far
    dyind = yind-pybuff; dxind = xind-pxbuff
    dist = math.sqrt(dyind**2+dxind**2)
    if dist > 5.0:
        warnings.warn(f'the most bright pixel and the xindex is too far for CR {ID}')
    
    # crop a patch of 32*32 but with its center at the peak
    xbuff = 16
    ybuff = 16
    ycrop = np.arange(yoff+dyind-ybuff,yoff+dyind+ybuff+1)
    xcrop = np.arange(xoff+dxind-xbuff,xoff+dxind+xbuff+1)
    cropcslc = cslc[ycrop,:][:,xcrop]

    # oversample this 32*32 patch by 32
    ovsFactor = 32
    cropcslc_ovs,ycrop_ovs,xcrop_ovs = oversample_slc(cropcslc,sampling=ovsFactor,y=ycrop,x=xcrop)

    # find the peak amplitude again in a 2 x 2 patch, it correspond to 
    # (2*ovsFactor) x (2*ovsFactor) in oversampled slc
    yoff2 = int(cropcslc_ovs.shape[0]/2)
    xoff2 = int(cropcslc_ovs.shape[1]/2)
    cropcslc2 = cropcslc_ovs[yoff2-ovsFactor:yoff2+ovsFactor+1,
                           xoff2-ovsFactor:xoff2+ovsFactor+1]
    yind2,xind2 = np.unravel_index(np.argmax(abs(cropcslc2), axis=None), cropcslc2.shape)
    dyind2 = yind2-ovsFactor; dxind2 = xind2-ovsFactor

    # crop a patch of 3x3 oversampled patch with center at the peak
    cropcslc2 = cropcslc_ovs[yoff2+dyind2-1:yoff2+dyind2+2,xoff2+dxind2-1:xoff2+dxind2+2]
    ycrop2 = ycrop_ovs[yoff2+dyind2-1:yoff2+dyind2+2]
    xcrop2 = xcrop_ovs[xoff2+dxind2-1:xoff2+dxind2+2]
    xxcrop2,yycrop2 = np.meshgrid(xcrop2,ycrop2)
    xxcrop2_f = xxcrop2.flatten()
    yycrop2_f = yycrop2.flatten()
    cropcslc2_f = cropcslc2.flatten()

    # Check if pixel values in a patch are non-NaN
    valid = ~(np.isnan(cropcslc2_f))
    count_valid = np.count_nonzero(valid)

    if count_valid == 0:
        _ypeak, _xpeak = [np.nan, np.nan]

    else:
        _ypeak,_xpeak = findCR(np.abs(cropcslc2_f[valid]),yycrop2_f[valid],xxcrop2_f[valid],
                            x_bound=[xcrop2[0],xcrop2[-1]],y_bound=[ycrop2[0],ycrop2[-1]],method="para")

    xpeak.append(_xpeak)
    ypeak.append(_ypeak)

df_filter['CSLC_xloc'] = xpeak
df_filter['CSLC_yloc'] = ypeak

In [ ]:
df_filter = df_filter.dropna()
df_filter

In [ ]:
#absloute geolocation error in range and azimuth after corrections
ALE_Rg = (df_filter['CSLC_xloc'] -  df_filter['CR_xloc'])
ALE_Az = (df_filter['CSLC_yloc'] - df_filter['CR_yloc'])

# Add to the dataframe
df_filter.loc[:,"ALE_Rg"] = ALE_Rg
df_filter.loc[:,"ALE_Az"] = ALE_Az

# write filtered database to file
df_filter.to_csv(f'filteredCR_ALE_{date}.csv', mode='a')

df_filter.head()

In [ ]:
#plotting ALE
fig, ax = plt.subplots(figsize=(8,6))
sc = ax.scatter(df_filter.ALE_Rg, df_filter.ALE_Az, s=200, c=df_filter['slen'], alpha=0.8, marker='o')
ax.legend(*sc.legend_elements(),facecolor='lightgray')
ax.get_legend().set_title('side length (m)')
for ii, txt in enumerate(df_filter.ID):
    ax.annotate(txt, (df_filter[df_filter['ID']==txt].ALE_Rg, df_filter[df_filter['ID']==txt].ALE_Az), color='orange')   #putting IDs in each CR
ax.grid(True)
#ax.set_xlim(-3,3)
#ax.set_ylim(-3,3)
ax.axhline(0, color='black')
ax.axvline(0, color='black')
ax.set_title(f'Ground Range: {np.round(np.nanmean(ALE_Rg), 3)} +/- {np.round(np.nanstd(ALE_Rg),3)} m, \
    Azimuth: {np.round(np.nanmean(ALE_Az),3)}, +/- {np.round(np.nanstd(ALE_Az),3)} m')
ax.set_xlabel('$\Delta x$ (m)')
ax.set_ylabel('$\Delta y$ (m)')
fig.suptitle(f'Absolute location error for {date}')
fig.savefig(f'ALE_geoSLC_{date}.png',dpi=300,bbox_inches='tight')

In [ ]:
ALE_Rg_Mean = np.round(np.nanmean(ALE_Rg), 3)
ALE_Az_Mean = np.round(np.nanmean(ALE_Az), 3)
ALE_Rg_Stdev = np.round(np.nanstd(ALE_Rg),3)
ALE_Az_Stdev = np.round(np.nanstd(ALE_Az),3)

summary = []
summary.append([date, ALE_Rg_Mean, ALE_Rg_Stdev, ALE_Az_Mean, ALE_Az_Stdev])
summary_df = pd.DataFrame(summary)
summary_df.to_csv('ALE.csv', mode='a', header=False)

In [ ]:
#load and plot mean summary file of entire burst record up to this date
df_summary = pd.read_csv('ALE.csv', header=None, usecols = [1, 2, 3, 4, 5])
df_summary.columns = ['date', 'ALE_Rg_Mean', 'ALE_Rg_Stdev', 'ALE_Az_Mean', 'ALE_Az_Stdev']

#plotting ALE
fig, ax = plt.subplots(figsize=(8,6))
sc = ax.scatter(df_summary.ALE_Rg_Mean, df_summary.ALE_Az_Mean, s=200, alpha=0.8, marker='o')
ax.grid(True)
ax.axhline(0, color='black')
ax.axvline(0, color='black')
ax.set_title(f'Mean round Range: {np.round(np.nanmean(df_summary.ALE_Rg_Mean), 3)} +/- {np.round(np.nanstd(df_summary.ALE_Rg_Mean),3)} m, \
    Mean Azimuth: {np.round(np.nanmean(df_summary.ALE_Az_Mean),3)}, +/- {np.round(np.nanstd(df_summary.ALE_Az_Mean),3)} m')
ax.set_xlabel('$\Delta x$ (m)')
ax.set_ylabel('$\Delta y$ (m)')
fig.suptitle(f'Absolute location error up to date {date}')
fig.savefig(f'ALE_mean_summary_upto_{date}.png',dpi=300,bbox_inches='tight')

In [ ]:
#load and plot summary file of entire burst record up to this date
all_files = glob.glob('filteredCR_ALE_*.csv')
df_all = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

#only proceed if these summary files are different
#i.e. there are multiple CRs for a given burst
if len(df_all) != len(df_summary):
    fig, ax = plt.subplots(figsize=(8,6))
    sc = ax.scatter(df_all.ALE_Rg, df_all.ALE_Az, s=200, c=df_all['slen'], alpha=0.8, marker='o')
    ax.legend(*sc.legend_elements(),facecolor='lightgray')
    ax.get_legend().set_title('side length (m)')
    ax.grid(True)
    ax.axhline(0, color='black')
    ax.axvline(0, color='black')
    ax.set_title(f'Ground Range: {np.round(np.nanmean(df_all.ALE_Rg), 3)} +/- {np.round(np.nanstd(df_all.ALE_Rg),3)} m, \
        Azimuth: {np.round(np.nanmean(df_all.ALE_Az),3)}, +/- {np.round(np.nanstd(df_all.ALE_Az),3)} m')
    ax.set_xlabel('$\Delta x$ (m)')
    ax.set_ylabel('$\Delta y$ (m)')
    fig.suptitle(f'Absolute location error for {date}')
    fig.savefig(f'ALE_summary_upto_{date}.png',dpi=300,bbox_inches='tight')